$$\textbf{PLN. Tarea 2: Minería de texto básica}$$
$$\textit{Y. Sarahi García Gozález}$$

In [83]:
from nltk.tokenize import TweetTokenizer
import nltk
import numpy as np
import pandas as pd
#clasificacion
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_recall_fscore_support,roc_auc_score

In [4]:
print("Tarea realizada en MacOs. \nLas versiones de las librerías y de python utilizadas fueron:\n")
from platform import python_version
print("Python version", python_version())
print("Numpy version", np.__version__)
print("NLTK version", nltk.__version__)

Tarea realizada en MacOs. 
Las versiones de las librerías y de python utilizadas fueron:

Python version 3.11.5
Numpy version 1.23.5
NLTK version 3.8.1


<font size=4.5 color='lightblue'>

$\textit{Ejercicio 2. Bolsas de Palabras, Bigramas y Emociones}$

$\textit{Preparación de texto, corpus y diccionarios}$

In [5]:

def get_text_from_file(path_corpus,path_truth):

    tr_txt=[]
    tr_labels=[]

    with open(path_corpus, "r") as f_corpus,open(path_truth, "r") as f_truth:
        for tweet in f_corpus:
            tr_txt += [tweet]
        for label in f_truth:
            tr_labels += [label]   
             
    return tr_txt, tr_labels


def create_corpus_from_text(text,tokenizer):
    corpus_palabras=[]
    for documento in text:
        corpus_palabras+=tokenizer.tokenize(documento)

    return corpus_palabras


def create_dic_freq(corpus,n):
    fdist = nltk.FreqDist(corpus)
    aux=[(fdist[key],key) for key in fdist]
    aux.sort()
    aux.reverse()
    aux=aux[:n]

    return aux

def create_dic_ranking(dic_freq):
    dict_indices=dict()
    cont = 0
    for weight, word in dic_freq:
        dict_indices[word]= cont
        cont+= 1

    return dict_indices

#Clasificador

parameters={'C':[0.05,0.12,0.25,0.5,1,2,4]} #parámetro de complejidad

#llamamos el clasificador SVM
#nota: cambié el max_iter y dual porque me aparecían una serie de warnings pero no cambiaron los resultados
svr=svm.LinearSVC(class_weight='balanced',dual='auto',max_iter=3000) 

#grid search recide el objeto de clasificacion
grid=GridSearchCV(estimator=svr,param_grid=parameters,n_jobs=8,scoring="f1_macro",cv=5)



In [6]:
#guardamos los textos de entrenamiento y validación
tr_txt,tr_labels=get_text_from_file("/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_train.txt","/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_train_labels.txt")
val_txt,val_labels=get_text_from_file("/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_val.txt","/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/practicas/03_practica/mex20_val_labels.txt")

In [7]:
#connvertimos a lista las etiquetas
tr_labels=list(map(int,tr_labels))
val_labels=list(map(int,val_labels))

In [8]:
#Tokenizamos
tokenizer=TweetTokenizer()
#Generamos el corpus
corpus_palabras=create_corpus_from_text(tr_txt,tokenizer)
#Generamos diccionario de frecuencias con las primeras 5 mil palabras
dict_freq=create_dic_freq(corpus_palabras,5000)
#Generamos diccionario de indices
dict_indices=create_dic_ranking(dict_freq)

In [47]:
#imprimimos algunos elementos del diccionario de indices
lista_indices = list(dict_indices.items())
for clave, valor in lista_indices[10:30]:
    print(clave, "-", valor)

me - 10
el - 11
en - 12
se - 13
es - 14
con - 15
? - 16
verga - 17
los - 18
madre - 19
por - 20
las - 21
" - 22
un - 23
te - 24
mi - 25
lo - 26
putas - 27
una - 28
... - 29


<font size=3.5 color='lightblue'>

1,2,3. Evalue BoW con pesado binario, de frecuencia y tfidf

$\textit{Bolsas de palabras con distintos equemas de pesado y sin normalizar}$

In [59]:
#________________________________PESADO_BINARIO________________________________#


#matriz_nxm de bolsa de palabras n=numero de textos, m=numero de palabras (5000)
def build_binary_bow(tr_txt,dic_freq,dic_indices):
    '''
    Función que genera la bolsa de palabras con pesado binario

    '''
    BoW=np.zeros((len(tr_txt),len(dic_freq)),dtype=int) #construimos bolsta de palabras en ceros
    cont_documento=0 #indice que recorre las FILAS

    for tr in tr_txt:#recorremos cada documento
        fdist_doc=nltk.FreqDist(tokenizer.tokenize(tr))#hacemos freqDist (tokenizado con el tokenizador ya definido) de cada documento
        for word in fdist_doc: #para cada palabra en el diccionario del documento
            if word in dic_indices: #si la palabra está en el corte final
                #AGREGAMOS un uno en el lugar correspondiente
                BoW[cont_documento,dic_indices[word]] = 1 

        cont_documento+=1

    return BoW

#________________________________PESADO_FRECUENCIA________________________________#

#matriz_nxm de bolsa de palabras n=numero de textos, m=numero de palabras (5000)
def build_freq_bow(tr_txt,dic_feq,dic_inices):
    BoW=np.zeros((len(tr_txt),len(dic_feq)),dtype=int) #construimos bolsta de palabras en ceros
    cont_documento=0 #indice que recorre las FILAS
 
    for tr in tr_txt:#recorremos cada documento
        fdist_doc=nltk.FreqDist(tokenizer.tokenize(tr))#hacemos freqDist (tokenizado con el tokenizador ya definido) de cada documento
        for word in fdist_doc: #para cada palabra en el diccionario del documento
            if word in dic_inices: #si la palabra está en el corte final
                #le asignamos su frecuencia
                BoW[cont_documento,dic_inices[word]] = fdist_doc[word]

        cont_documento+=1

    return BoW

#________________________________PESADO_TFIDF________________________________#

#primero calculamos el num de documentos en que aparece cada palabra
def df(tr_txt, dic_indices):
    df = np.zeros(5000, dtype=int)

    # comp list para las frecuencias de cada documento
    fdist_docs = [nltk.FreqDist(tokenizer.tokenize(tr)) for tr in tr_txt]

    #iteramos sobre cada palbra
    for word in dic_indices:
        #obtenemos el indice de la palabra
        index = dic_indices[word] 
        for j,fdist_doc in enumerate(fdist_docs): #iteramos osbre cada documento
            if word in fdist_doc: #si la palabra está en el documento actual
                df[index] += 1 #agredamos uno al conador de esa palabra

    return df

#
def build_tfidf_bow(tr_txt, dic_freq, dic_indices):
    BoW = build_freq_bow(tr_txt, dic_freq, dic_indices) #la bolsa de palabras de frecuencias es el tf
    df_list = df(tr_txt, dic_indices) #llamamos a la duncion df
    N = len(tr_txt)
   
    for i in range(N):
        for j,df_val in enumerate(df_list): 
            if df_val!=0: #si df es distinto de ceo (puede ser cero para el conjunto de validacion, por ejemplo que contiene menos lementos )
                BoW[i][j] *= np.log(N / df_val) #tfxdf

    return BoW

In [54]:
dict_indices

{',': 0,
 'de': 1,
 'que': 2,
 '.': 3,
 'la': 4,
 'a': 5,
 'y': 6,
 '!': 7,
 'no': 8,
 '@USUARIO': 9,
 'me': 10,
 'el': 11,
 'en': 12,
 'se': 13,
 'es': 14,
 'con': 15,
 '?': 16,
 'verga': 17,
 'los': 18,
 'madre': 19,
 'por': 20,
 'las': 21,
 '"': 22,
 'un': 23,
 'te': 24,
 'mi': 25,
 'lo': 26,
 'putas': 27,
 'una': 28,
 '...': 29,
 'putos': 30,
 'para': 31,
 '😂': 32,
 'si': 33,
 'ya': 34,
 'como': 35,
 'su': 36,
 'pero': 37,
 'tu': 38,
 'loca': 39,
 'le': 40,
 'más': 41,
 'No': 42,
 'del': 43,
 'gorda': 44,
 'al': 45,
 'bien': 46,
 'A': 47,
 '¿': 48,
 'Y': 49,
 'son': 50,
 'Me': 51,
 'o': 52,
 'feas': 53,
 'cuando': 54,
 'Que': 55,
 ':': 56,
 'yo': 57,
 'les': 58,
 'porque': 59,
 'ni': 60,
 'está': 61,
 'ser': 62,
 'estoy': 63,
 'sus': 64,
 'todos': 65,
 'esta': 66,
 'puta': 67,
 'Ya': 68,
 'todo': 69,
 'pinche': 70,
 'puto': 71,
 'tan': 72,
 'Si': 73,
 'La': 74,
 'qué': 75,
 '…': 76,
 'eso': 77,
 'muy': 78,
 'soy': 79,
 'hasta': 80,
 'así': 81,
 '¡': 82,
 '<URL>': 83,
 'mamar': 84,


In [11]:
#generamos las bolsas de palabrascon pesado binario  de training y de validación 
binary_bow_tr=build_binary_bow(tr_txt,dict_freq,dict_indices)
binary_boW_validacion=build_binary_bow(val_txt,dict_freq,dict_indices)

In [12]:
#generamos las bolsas de palabras con pesado frecuencia de training y de validación 
frequency_bow_tr=build_freq_bow(tr_txt,dict_freq,dict_indices)
frequency_boW_validacion=build_freq_bow(val_txt,dict_freq,dict_indices)

In [60]:
#generamos las bolsas de palabras con pesado tfidf de training y de validación 
tfidf_bow_tr=build_tfidf_bow(tr_txt,dict_freq,dict_indices)
tfidf_boW_validacion=build_tfidf_bow(val_txt,dict_freq,dict_indices)

* Diferencias en bolsa de palabras

veamos la diferencia entre los pesos con cada esquema para un caso particular:

In [61]:
#el token ""@USUARIO"" corresponde al lugar 9 en el diccionario de frecuencias 
x=dict_indices.get("@USUARIO")
print("el token ""@USUARIO"" corresponde al lugar",x, "en el diccionario de frecuencias ")
print("el primer tweet de los datos (es decir el documento 0) contiene tres veces este token")
print("El peso w_(0,9) es:")
print("con pesado binario:",binary_bow_tr[0][9])
print("con pesado de frecnuencia:", frequency_bow_tr[0][9])
print("con pesado de tfidf:", tfidf_bow_tr[0][9])

el token @USUARIO corresponde al lugar 9 en el diccionario de frecuencias 
el primer tweet de los datos (es decir el documento 0) contiene tres veces este token
El peso w_(0,9) es:
con pesado binario: 1
con pesado de frecnuencia: 3
con pesado de tfidf: 5


$\textit{Clasificador}$

In [15]:
#llamamos al clasificador y le pasamos la bolsa de palabras binaria y las etiqutas
grid.fit(binary_bow_tr,tr_labels)
#predicción
y_pred_binary=grid.predict(binary_boW_validacion)


In [16]:
#llamamos al clasificador y le pasamos la bolsa de palabras de frecuencia y las etiqutas
grid.fit(frequency_bow_tr,tr_labels)
#predicción
y_pred_frequency=grid.predict(frequency_boW_validacion)

/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/.conda/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [62]:
#llamamos al clasificador y le pasamos la bolsa de palabras TFIDF y las etiqutas
grid.fit(tfidf_bow_tr,tr_labels)
#predicción
y_pred_tfidf=grid.predict(tfidf_boW_validacion)

/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/.conda/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/.conda/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/.conda/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/.conda/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/

In [18]:
#medidas, reporte de clasificación,matriz de confusión
print("_____________Binary BoW_____________")
print(confusion_matrix(val_labels,y_pred_binary))
print(metrics.classification_report(val_labels,y_pred_binary))

_____________Binary BoW_____________
[[356  62]
 [ 49 120]]
              precision    recall  f1-score   support

           0       0.88      0.85      0.87       418
           1       0.66      0.71      0.68       169

    accuracy                           0.81       587
   macro avg       0.77      0.78      0.77       587
weighted avg       0.82      0.81      0.81       587



In [19]:
#medidas, reporte de clasificación,matriz de confusión
print("_____________Frequency BoW_____________")
print(confusion_matrix(val_labels,y_pred_frequency))
print(metrics.classification_report(val_labels,y_pred_frequency))

_____________Frequency BoW_____________
[[362  56]
 [ 45 124]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       418
           1       0.69      0.73      0.71       169

    accuracy                           0.83       587
   macro avg       0.79      0.80      0.79       587
weighted avg       0.83      0.83      0.83       587



In [63]:
#medidas, reporte de clasificación,matriz de confusión
print("_____________TfiDf BoW_____________")
print(confusion_matrix(val_labels,y_pred_tfidf))
print(metrics.classification_report(val_labels,y_pred_tfidf))

_____________TfiDf BoW_____________
[[356  62]
 [ 71  98]]
              precision    recall  f1-score   support

           0       0.83      0.85      0.84       418
           1       0.61      0.58      0.60       169

    accuracy                           0.77       587
   macro avg       0.72      0.72      0.72       587
weighted avg       0.77      0.77      0.77       587



<font size=2.5 color='lightblue'>

4,5,6. Evalue BoW con pesado binario, de frecuencia y tfidf con normalizado l2

$\textit{Bolsas de palabras con distintos equemas de pesado con normalizado L2}$

In [21]:
def normalizacion_dos(X):
    norm = np.linalg.norm(X, ord=2, axis=1)  # Calcula la norma L2 de cada fila
    norm = np.where(norm == 0, 1, norm) 
    norm = norm.reshape(-1, 1)  # Reshape para que la división sea compatible
    return X / norm  # Divide cada elemento de X por la norma correspondiente 

In [22]:
#normalizamos las bolsas de palabrascon pesado binario  de training y de validación 
binary_bow_tr_norm=normalizacion_dos(binary_bow_tr)
binary_boW_validacion_norm=normalizacion_dos(binary_boW_validacion)

In [23]:
#normalizamos las bolsas de palabrascon pesado frecuencia  de training y de validación 
frequency_bow_tr_norm=normalizacion_dos(frequency_bow_tr)
frequency_boW_validacion_norm=normalizacion_dos(frequency_boW_validacion)

In [64]:
#normalizamos las bolsas de palabrascon pesado tfidf  de training y de validación 
tfidf_bow_tr_norm=normalizacion_dos(tfidf_bow_tr)
tfidf_boW_validacion_norm=normalizacion_dos(tfidf_boW_validacion)

* Diferencias en bolsa de palabras

veamos la diferencia entre los pesos con cada esquema para un caso particular:

In [65]:
#el token ""@USUARIO"" corresponde al lugar 9 en el diccionario de frecuencias 
x=dict_indices.get("@USUARIO")
print("el token ""@USUARIO"" corresponde al lugar",x, "en el diccionario de frecuencias ")
print("el primer tweet de los datos (es decir el documento 0) contiene tres veces este token")
print("El peso w_(0,9) es:")
print("con pesado binario:",binary_bow_tr_norm[0][9])
print("con pesado de frecnuencia:", frequency_bow_tr_norm[0][9])
print("con pesado de tfidf:", tfidf_bow_tr_norm[0][9])

el token @USUARIO corresponde al lugar 9 en el diccionario de frecuencias 
el primer tweet de los datos (es decir el documento 0) contiene tres veces este token
El peso w_(0,9) es:
con pesado binario: 0.20412414523193154
con pesado de frecnuencia: 0.48666426339228763
con pesado de tfidf: 0.21128856368212914


$\textit{Clasificador}$

In [26]:
#llamamos al clasificador y le pasamos la bolsa de palabras binaria y las etiqutas
grid.fit(binary_bow_tr_norm,tr_labels)
#predicción
y_pred_binary_norm=grid.predict(binary_boW_validacion_norm)

In [27]:
#llamamos al clasificador y le pasamos la bolsa de palabras de frecuencia y las etiqutas
grid.fit(frequency_bow_tr_norm,tr_labels)
#predicción
y_pred_frequency_norm=grid.predict(frequency_boW_validacion_norm)

In [66]:
#llamamos al clasificador y le pasamos la bolsa de palabras TFIDF y las etiqutas
grid.fit(tfidf_bow_tr_norm,tr_labels)
#predicción
y_pred_tfidf_norm=grid.predict(tfidf_boW_validacion_norm)

In [29]:
#medidas, reporte de clasificación,matriz de confusión
print("_____________Binary BoW L2 normalization_____________")
print(confusion_matrix(val_labels,y_pred_binary_norm))
print(metrics.classification_report(val_labels,y_pred_binary_norm))

_____________Binary BoW L2 normalization_____________
[[356  62]
 [ 47 122]]
              precision    recall  f1-score   support

           0       0.88      0.85      0.87       418
           1       0.66      0.72      0.69       169

    accuracy                           0.81       587
   macro avg       0.77      0.79      0.78       587
weighted avg       0.82      0.81      0.82       587



In [30]:
#medidas, reporte de clasificación,matriz de confusión
print("_____________Frequency BoW L2 normalization_____________")
print(confusion_matrix(val_labels,y_pred_frequency_norm))
print(metrics.classification_report(val_labels,y_pred_frequency_norm))

_____________Frequency BoW L2 normalization_____________
[[358  60]
 [ 38 131]]
              precision    recall  f1-score   support

           0       0.90      0.86      0.88       418
           1       0.69      0.78      0.73       169

    accuracy                           0.83       587
   macro avg       0.79      0.82      0.80       587
weighted avg       0.84      0.83      0.84       587



In [67]:
#medidas, reporte de clasificación,matriz de confusión
print("_____________TfiDf BoW L2 normalization_____________")
print(confusion_matrix(val_labels,y_pred_tfidf_norm))
print(metrics.classification_report(val_labels,y_pred_tfidf_norm))


_____________TfiDf BoW L2 normalization_____________
[[356  62]
 [ 40 129]]
              precision    recall  f1-score   support

           0       0.90      0.85      0.87       418
           1       0.68      0.76      0.72       169

    accuracy                           0.83       587
   macro avg       0.79      0.81      0.80       587
weighted avg       0.83      0.83      0.83       587



<font size=2.5 color='lightblue'>

7. Ponga una tabla comparativa a modo de resumen con las seis entradas anteriores.

In [68]:
lista_predicciones=[y_pred_binary,y_pred_binary_norm,y_pred_frequency,y_pred_frequency_norm,y_pred_tfidf,y_pred_tfidf_norm]
lista_pesados=["Pesado binario","Pesado binario normlizado","Pesado frecuencia","Pesado frecuencia normalizado","Pesado tfidf","Pesado tfidf normalizado"]
for predict, pesado in zip(lista_predicciones, lista_pesados):
    m = metrics.classification_report(val_labels, predict)
    accuracy = float(m.split('\n')[-2].split()[2])
    precision_0 = float(m.split('\n')[2].split()[1])  
    precision_1 = float(m.split('\n')[3].split()[1])  
    
    print(pesado, "\n Accuracy", accuracy, "\n Precision (0)",precision_0,"\n Precision (1)",precision_1 )

Pesado binario 
 Accuracy 0.82 
 Precision (0) 0.88 
 Precision (1) 0.66
Pesado binario normlizado 
 Accuracy 0.82 
 Precision (0) 0.88 
 Precision (1) 0.66
Pesado frecuencia 
 Accuracy 0.83 
 Precision (0) 0.89 
 Precision (1) 0.69
Pesado frecuencia normalizado 
 Accuracy 0.84 
 Precision (0) 0.9 
 Precision (1) 0.69
Pesado tfidf 
 Accuracy 0.77 
 Precision (0) 0.83 
 Precision (1) 0.61
Pesado tfidf normalizado 
 Accuracy 0.83 
 Precision (0) 0.9 
 Precision (1) 0.68


El accuracy más alto se obtene para el pesado *frecuencia normalizado*, con 0.84. Como tenemos un conjunto desbalanceado buscaremos maximizar el recall para la clase minoritaria (etiqueta 1) para capturar la mayor cantidad posible de casos positivos verdaderos, así que comparremos con la el siguiente pesado con mayor accuracy que es *Pesado tfidf normalizado*. Notemos a precisión para la etiqueta 0 es la misma: 0.9, miestras que para la etiqueta uno varia un poco (0.1), siendo mayor para Frecuecia.

In [69]:
lista_predicciones=[y_pred_frequency_norm,y_pred_tfidf_norm]
lista_pesados=["Pesado frecuencia normalizado","Pesado tfidf normalizado"]
for predict, pesado in zip(lista_predicciones, lista_pesados):
    m = metrics.classification_report(val_labels, predict)
    accuracy = float(m.split('\n')[-2].split()[2])
    precision_0 = float(m.split('\n')[2].split()[2])  
    precision_1 = float(m.split('\n')[3].split()[2])  
    
    print(pesado, "\n Accuracy", accuracy, "\n Recall (0)",precision_0,"\n Recall (1)",precision_1 )

Pesado frecuencia normalizado 
 Accuracy 0.84 
 Recall (0) 0.86 
 Recall (1) 0.78
Pesado tfidf normalizado 
 Accuracy 0.83 
 Recall (0) 0.85 
 Recall (1) 0.76


Así que el mejor resultado se obtuvo con pesado defrecuencia y normalización L2, mayor accuracy, mayor precision e incluso mayor recall para la etiqueta minoritaria. 

<font size=2.5 color='lightblue'>

8. De las configuraciones anteriores elija la mejor y evalúela con más y menos términos (e.g., 1000 y 7000). Ponga una tabla dónde compare las tres configuraciones.


* 1000

In [75]:
#Generamos diccionario de indices y de frecuencias haciendo corte en las mil primeras palabras
dict_freq_1000=create_dic_freq(corpus_palabras,1000)
dict_indices_1000=create_dic_ranking(dict_freq_1000)

#Bolsas de palabras con pesado frecuencia de training y de validación usando los diccionarios anteriores
bow_tr_1000=build_freq_bow(tr_txt,dict_freq_1000,dict_indices_1000)
boW_validacion_1000=build_freq_bow(val_txt,dict_freq_1000,dict_indices_1000)

#normalizamos 
bow_tr_norm_1000=normalizacion_dos(bow_tr_1000)
boW_validacion_norm_1000=normalizacion_dos(boW_validacion_1000)

In [76]:
#llamamos al clasificador y le pasamos la bolsa de palabras 
grid.fit(bow_tr_norm_1000,tr_labels)
#predicción
y_pred_1000=grid.predict(boW_validacion_norm_1000)

In [79]:
#medidas, reporte de clasificación,matriz de confusión
print("_____________Frequency BoW (1000) L2 normalization_____________")
print(confusion_matrix(val_labels,y_pred_1000))
print(metrics.classification_report(val_labels,y_pred_1000))

_____________Frequency BoW (1000) L2 normalization_____________
[[346  72]
 [ 39 130]]
              precision    recall  f1-score   support

           0       0.90      0.83      0.86       418
           1       0.64      0.77      0.70       169

    accuracy                           0.81       587
   macro avg       0.77      0.80      0.78       587
weighted avg       0.83      0.81      0.82       587



* 7000

In [77]:
#Generamos diccionario de indices y de frecuencias haciendo corte en las mil primeras palabras
dict_freq_7000=create_dic_freq(corpus_palabras,7000)
dict_indices_7000=create_dic_ranking(dict_freq_7000)

#Bolsas de palabras con pesado frecuencia de training y de validación usando los diccionarios anteriores
bow_tr_7000=build_freq_bow(tr_txt,dict_freq_7000,dict_indices_7000)
boW_validacion_7000=build_freq_bow(val_txt,dict_freq_7000,dict_indices_7000)

#normalizamos 
bow_tr_norm_7000=normalizacion_dos(bow_tr_7000)
boW_validacion_norm_7000=normalizacion_dos(boW_validacion_7000)

In [78]:
#llamamos al clasificador y le pasamos la bolsa de palabras 
grid.fit(bow_tr_norm_7000,tr_labels)
#predicción
y_pred_7000=grid.predict(boW_validacion_norm_7000)

In [80]:
#medidas, reporte de clasificación,matriz de confusión
print("_____________Frequency BoW (7000) L2 normalization_____________")
print(confusion_matrix(val_labels,y_pred_7000))
print(metrics.classification_report(val_labels,y_pred_7000))

_____________Frequency BoW (7000) L2 normalization_____________
[[359  59]
 [ 39 130]]
              precision    recall  f1-score   support

           0       0.90      0.86      0.88       418
           1       0.69      0.77      0.73       169

    accuracy                           0.83       587
   macro avg       0.79      0.81      0.80       587
weighted avg       0.84      0.83      0.84       587



* Comparación 

In [82]:
lista_predicciones=[y_pred_1000,y_pred_frequency_norm,y_pred_7000]
lista_pesados=["1000","5000","7000"]
print("Bolsa depalabras con pesado de frecuencia y normalización L2\n")
for predict, pesado in zip(lista_predicciones, lista_pesados):
    m = metrics.classification_report(val_labels, predict)
    accuracy = float(m.split('\n')[-2].split()[2])
    precision_0 = float(m.split('\n')[2].split()[1])  
    precision_1 = float(m.split('\n')[3].split()[1])  
    
    print("palabras tomadas para BoW:",pesado, "\n Accuracy", accuracy, "\n Precision (0)",precision_0,"\n Precision (1)",precision_1 )

Bolsa depalabras con pesado de frecuencia y normalización L2

palabras tomadas para BoW: 1000 
 Accuracy 0.83 
 Precision (0) 0.9 
 Precision (1) 0.64
palabras tomadas para BoW: 5000 
 Accuracy 0.84 
 Precision (0) 0.9 
 Precision (1) 0.69
palabras tomadas para BoW: 7000 
 Accuracy 0.84 
 Precision (0) 0.9 
 Precision (1) 0.69


<font size=2.5 color='lightblue'>

9. Utilice el recurso léxico del Consejo Nacional de Investigación de Canadá llamado "EmoLex" (https://www.saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) para construir una "Bolsa de Emociones" de los Tweets de agresividad (Debe usar EmoLex en Español). Para esto, una estrategia sencilla sería enmascarar cada palabra con su emoción, y después construir la Bolsa de Emociones (BoE).


* Descargamos emolex en español

In [23]:
def read_file_lines(file_path):
    lines = []
    with open(file_path, "r") as f:
        for line in f:
            lines.append(line.strip().split())
    return lines


In [24]:
emolex_sp=read_file_lines("/Users/ely/Documents/Maestria/segundo_semestre/cimat2023-1/lenguaje/NRC-Emotion-Lexicon/OneFilePerLanguage/Spanish-NRC-EmoLex.txt")

In [25]:
emolex_sp

[['English',
  'Word',
  'anger',
  'anticipation',
  'disgust',
  'fear',
  'joy',
  'negative',
  'positive',
  'sadness',
  'surprise',
  'trust',
  'Spanish',
  'Word'],
 ['aback', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'detras'],
 ['abacus', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', 'abaco'],
 ['abandon', '0', '0', '0', '5', '0', '6', '0', '7', '0', '0', 'abandonar'],
 ['abandoned', '1', '0', '0', '1', '0', '1', '0', '1', '0', '0', 'abandonado'],
 ['abandonment', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', 'abandono'],
 ['abate', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'disminuir'],
 ['abatement',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  'disminucion'],
 ['abba', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', 'abba'],
 ['abbot', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', 'abad'],
 ['abbreviate', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'abreviar'],
 ['abbreviation',
  '0',
  '0',
  '0',
  '0',
  '

* enmascaramos cada palabra del diccionario con su emocion

<font size=2.5 color='lightblue'>

10. Evalúa tú BoE clasificando con SVM. Ponga una tabla comparativa a modo de resumen con los tres pesados, normalize cada uno si lo cree conveniente.

<font size=4.5 color='lightblue'>

$\textit{Ejercicio 3. Recurso Línguistico de Emociones Mexicano}$

<font size=2.5 color='lightblue'>

11. Utiliceelrecursoléxicollamado"SpanishEmotionLexicon(SEL)"delDr.GrigoriSidorov, profesor del Centro de Investigación en Computación (CIC) del Instituto Politécnico Nacional (http://www.cic.ipn.mx/∼sidorov/), para enmascarar cada palabra con su emo- ción, y después construir la Bolsa de Emociones con algún pesado (e.g., binario, tf, tfidf). Proponga alguna estrategia para incorporar el "valor" del "Probability Factor of Affective use" en su representación vectorial del documento. Evalúa y escribe una tabla compara- tiva a modo de resumen con al menos tres pesados: binario, frecuencia, tfidf. Normalize cada pesado según lo crea conveniente.


<font size=2.5 color='lightblue'>

12. En un comentario aparte, discuta sobre la estrategía que utilizó para incorporar el "Probability Factor of Affective use". No más de 5 renglones.

<font size=4.5 color='lightblue'>

$\textit{Ejercicio 4. ¿Podemos mejorar con Bigramas?}$


<font size=2.5 color='lightblue'>

13. Hacer un experimento dónde concatene una buena BoW según sus experimentos anteri- ores con otra BoW construida a partir de los 1000 bigramas más frecuentes.


<font size=2.5 color='lightblue'>

14. Hacer un experimento con las Bolsas de Emociones, Bolsa de Palabras y Bolsa de Bi- gramas; usted elige las dimensionalidades. Para construir la representación final del documento utilice la concatenación de las representaciones según sus observaciones (e.g., Bolsa de Palabras + Bolsa de Bigramas + Bolsa de Sentimientos de Canadá + Bolsa de Sentimientos de Grigori), y aliméntelas a un SVM.


<font size=2.5 color='lightblue'>

15. Elabore conclusiones sobre toda esta Tarea, incluyendo observaciones, comentarios y posibles mejoras futuras. Discuta el comportamiento de la BoW de usar solo palabras a integrar bigramas, y luego a integrar todo ¿ayudó? o ¿empeoró?. Discuta también brevemente el costo computacional de los experimentos ¿Valió la Pena tener todo?. Sea breve: todo en NO más de dos párrafos.